# Let the challenge begin

**Notes on data** 

- 5 EEG derivations sampled at 250Hz
- 3 Accelerometers derivations sampled at 50Hz
- Sleep epoch = 30 sec
- hypnogram = succession of the sleep stages (0...5)

**General info sleep**
- Sleep stages = (N1, N2) = light sleep, N3 = deep sleep, REM
- Low frequency power: N3 > N2 > N1-REM-Wake

**Wake**
- During Wake epoch alpha waves are clearly visible on the F-O derivation
- Movement occured mainly during wake periods, noisy signals during movement
- Alpha wave frequency ranges between 8 and 13 hertz = wake, relaxed
**N1**
- Theta waves freq betw 4 and 8 Hz = N1, N2

**N2**
- On N2 epoch, power in the spindle range is much higher on frontal-frontal channels
- Theta waves freq betw 4 and 8 Hz = N1, N2
- During N2, sleep spindles (fast rythm between 12-14Hz which last between 0.5 up to 2 seconds) are more visible on the Frontal-frontal derivation

**N3**
- On N3 epoch, we can see more power in the low frequencies
- Delta waves freq betw 1 and 4 Hz = N3

**REM**
- REM sleep distinguishable with steady EEG and eyes movement which can be seen when looking at Frontal-occipital vs frontal-frontal derivation.
- The EEG power increases in the low-frequency band when the sleep stage change from REM to NREM sleep stages
- REM epoch have more steady EEG

**Formulas**
- Spectrogram are the time-frequency matrix z = P(t, f)
- Spectrum correspond to the curves y = P(frequency)
- Average Spectrum can therefore be computed as the mean of spectromgram over a specified period 

**Links**
https://opentext.wsu.edu/psych105/chapter/stages-of-sleep/
https://www.sleepfoundation.org/how-sleep-works/alpha-waves-and-sleep
https://centralesupelec.edunao.com/pluginfile.php/242107/course/section/36663/Challenge%20Data%20Dreem-1.pdf
https://centralesupelec.edunao.com/pluginfile.php/242107/course/section/36663/entropy-18-00272.pdf



In [24]:
cd "/kaggle/input/dreem-automated-sleep-staging/"

/kaggle/input/dreem-automated-sleep-staging


In [25]:
!pip install lspopt
!pip install skorch
!pip install cuda-python

directory = "/kaggle/input/dreem-automated-sleep-staging/"
out_directory = '/kaggle/working/'

#directory = './'
#out_directory = './'

#directory = '/kaggle/input/files-ml/'
#out_directory = '/kaggle/working/'

In [26]:
import numpy as np
import matplotlib as mpl
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from os import listdir
from random import randint
import random as rd
import os 

from lspopt import spectrogram_lspopt
from scipy.signal import spectrogram

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import KFold

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, confusion_matrix

import torch.optim as optim
from skorch import NeuralNetClassifier
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import torchaudio.transforms as T

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import ParameterGrid, GridSearchCV

import joblib

In [27]:
frequency_bands = {
        "delta": [0.5, 4],
        "theta": [4, 8],
        "alpha": [8, 12],
       "sigma": [12, 16],
       "beta": [16, 30]
    }

EEG_FS = 250
ACC_FS = 50 
epoch_s = 30
n_EEG = 5
n_ACC = 3

hypnograms = pd.read_csv(directory + 'targets_train.csv')

In [28]:
#plt.imshow(plt.imread(directory + 'corr_sleep_stages.png'))

## Functions 

In [29]:
def get_average_spectrum_for_epochs(eeg,epochs):
    """
    Return the average power in each of the fourier bin for several epochs.
    """
    psds = []
    for epoch in epochs:
        idx_start,idx_end = 250 * 30 * epoch,250 * 30 * (epoch + 1)
        freqs,t,psd = spectrogram_lspopt(np.clip(eeg[idx_start:idx_end],-150,150),250,nperseg = 1000)
        psds += [np.mean(psd ** 2,1)]
    return freqs,np.array(psds).mean(0)


## Function to plot N sleep epochs for a specific stage

def random_sleep_epoch(N, sleep_stage, hypnogram) :
    k = 0
    a = randint(0,len(hypnogram))
    epochs = []
    while k < N:
        if hypnogram[a] == sleep_stage :
            epochs.append(a)
            k += 1
            a = randint(0,len(hypnogram))
        else :
            a = randint(0,len(hypnogram))
    eeg_ff = np.load('sample/sample/f8_f7.npy')
    for epoch in epochs : 
        t0 = epoch*epoch_s*EEG_FS
        eeg_short = eeg_ff[t0:t0+(epoch_s*EEG_FS)]
        plt.figure(figsize=(25, 8))
        plt.plot(eeg_short)
        plt.ylim([-200, 200])
        plt.xlim(0,len(eeg_short))
        plt.show()

def plot_spectrogram(
    data,
    sf,
    hypno=None,
    win_sec=30,
    fmin=0.5,
    fmax=25,
    trimperc=2.5,
    cmap="RdBu_r",
    vmin=None,
    vmax=None,
):
    # Safety checks
    assert isinstance(data, np.ndarray), "Data must be a 1D NumPy array."
    assert isinstance(sf, (int, float)), "sf must be int or float."
    assert data.ndim == 1, "Data must be a 1D (single-channel) NumPy array."
    assert isinstance(win_sec, (int, float)), "win_sec must be int or float."
    assert isinstance(fmin, (int, float)), "fmin must be int or float."
    assert isinstance(fmax, (int, float)), "fmax must be int or float."
    assert fmin < fmax, "fmin must be strictly inferior to fmax."
    assert fmax < sf / 2, "fmax must be less than Nyquist (sf / 2)."
    assert isinstance(vmin, (int, float, type(None))), "vmin must be int, float, or None."
    assert isinstance(vmax, (int, float, type(None))), "vmax must be int, float, or None."
    if vmin is not None:
        assert isinstance(vmax, (int, float)), "vmax must be int or float if vmin is provided"
    if vmax is not None:
        assert isinstance(vmin, (int, float)), "vmin must be int or float if vmax is provided"

    # Calculate multi-taper spectrogram
    nperseg = int(win_sec * sf)
    assert data.size > 2 * nperseg, "Data length must be at least 2 * win_sec."
    f, t, Sxx = spectrogram_lspopt(data, sf, nperseg=nperseg, noverlap=0)
    Sxx = 10 * np.log10(Sxx)  # Convert uV^2 / Hz --> dB / Hz

    # Select only relevant frequencies (up to 30 Hz)
    good_freqs = np.logical_and(f >= fmin, f <= fmax)
    Sxx = Sxx[good_freqs, :]
    f = f[good_freqs]
    t /= 3600  # Convert t to hours

    # Normalization
    if vmin is None:
        vmin, vmax = np.percentile(Sxx, [0 + trimperc, 100 - trimperc])
        norm = Normalize(vmin=vmin, vmax=vmax)
    else:
        norm = Normalize(vmin=vmin, vmax=vmax)

    if hypno is None:
        fig, ax = plt.subplots(nrows=1, figsize=(3, 1))
        im = ax.pcolormesh(t, f, Sxx, norm=norm, cmap=cmap, antialiased=True, shading="auto")
        ax.set_xlim(0, t.max())
        plt.setp([ax.get_xticklines() + ax.get_yticklines() + ax.get_xgridlines() + ax.get_ygridlines()],antialiased=False)
        mpl.rcParams['text.antialiased']=False
        fig.tight_layout(pad=0)
        return fig

def split_train_test(records, split=0.5) : 
    rd.seed(1234)
    rd.shuffle(records)
    k = int(len(records)*split)
    return records[:k],records[k:]


def spectral_power(data,n,fs) :
    for i in range (n) :
        sfreqs,t,psd = spectrogram(data[:,i,:], fs, nperseg = 1000,noverlap = 750)
        psd = np.mean(np.abs(psd),-1)
        l = []
        for name, freqband in frequency_bands.items():
            spec_power = psd[:,(sfreqs >= freqband[0]) & (sfreqs < freqband[1])]
            spec_power = np.sum(spec_power, 1)
            l.append(spec_power / np.sum(psd,1))
        matrice = np.array(l) 
        matrice = np.vstack((matrice, np.array([np.mean(data[k,i,:]) for k in range (len(data))]).T))
        matrice = np.vstack((matrice, np.array([np.std(data[k,i,:]) for k in range (len(data))]).T))
        if i == 0:
            complete_array = matrice 
        else :
            complete_array = np.vstack((complete_array,matrice))
    return(complete_array.T)


def only_stats(data,n) : 
    for i in range (n) :
        matrice = np.array([np.mean(data[k,i,:]) for k in range (len(data))]).T
        matrice = np.vstack((matrice, np.array([np.std(data[k,i,:]) for k in range (len(data))]).T))
        if i == 0:
            complete_array = matrice 
        else :
            complete_array = np.vstack((complete_array,matrice))
    return(complete_array.T)


def correlations(record, data, n) :
    corr = [0]*n
    for i in range (n) :
        corr[i] = [0]*n
        for j in range (n) :
            corr[i][j] = np.corrcoef(data[record,i,:], data[record,j,:])[0][1]
        print(i+1,corr[i]) 
    return corr  

def whitening(X):  
    cov = np.cov(X)
    d, E = np.linalg.eigh(cov)
    D = np.diag(d)
    D_inv = np.sqrt(np.linalg.inv(D))
    X_whiten = np.dot(E, np.dot(D_inv, np.dot(E.T, X)))
    return X_whiten

def normalize_data(eeg_array):
    """normalize signal between 0 and 1"""

    normalized_array = np.clip(eeg_array, -250, 250)
    normalized_array = normalized_array / 250

    return normalized_array


class EegEpochDataset(Dataset):
    """EEG Epochs dataset."""

    def __init__(self, x_data, y_data, transform=None):
        """
        Args:
            x_data (numpy array): Numpy array of input data.
            y_data (list of numpy array): Sleep Stages
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.y_data = y_data
        self.x_data = x_data
        self.transform = transform

        self.x_data = normalize_data(x_data)

    def __len__(self):
        return len(self.y_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        signal = np.expand_dims(self.x_data[idx], axis=0)
        stage = self.y_data[idx]

        if self.transform:
            signal = self.transform(signal)

        return signal, stage
    
class EegEpochDataset_test(Dataset):
    """EEG Epochs dataset."""

    def __init__(self, x_data, transform=None):
        """
        Args:
            x_data (numpy array): Numpy array of input data.
            y_data (list of numpy array): Sleep Stages
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.x_data = x_data
        self.transform = transform

        self.x_data = normalize_data(x_data)
        
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        signal = np.expand_dims(self.x_data[idx], axis=0)

        if self.transform:
            signal = self.transform(signal)

        return signal


class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

def weights_init(m):
    if isinstance(m, nn.Conv1d) or isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)

class SingleChannelConvNet(nn.Module):

    def __init__(self):
        super(SingleChannelConvNet, self).__init__()
        self.conv_a = nn.Conv1d(1, 8, 5, stride=5)
        self.batchnorm_a = nn.BatchNorm1d(8, eps=0.001, momentum=0.99)
        self.conv_b = nn.Conv1d(8, 32, 5, stride=3)
        self.batchnorm_b = nn.BatchNorm1d(32, eps=0.001, momentum=0.99)
        self.conv_c = nn.Conv1d(32, 64, 3, stride=3)
        self.batchnorm_c= nn.BatchNorm1d(64, eps=0.001, momentum=0.99)
        self.conv_d = nn.Conv1d(64, 128, 3, stride=2)
        self.batchnorm_d= nn.BatchNorm1d(128, eps=0.001, momentum=0.99)
        self.conv_e = nn.Conv1d(128, 256, 3, stride=1)
        self.batchnorm_e= nn.BatchNorm1d(256, eps=0.001, momentum=0.99)
       
        self.max_pool_a = nn.MaxPool1d(5, stride=1)
        self.max_pool_prime = nn.MaxPool1d(3, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        # Size of a layer after convolution : (-W - F + 2P)/S +1 
        # / size of conv a after max pool : (8 - 2)/1 +1 = 7
        # self.avg_pool=torch.nn.AvgPool1d(kernel_size=256)

        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(118784, 256)
        self.fc2 = nn.Linear(256, 5)
        self.fc_lin = nn.Linear(256,5)

    def forward(self, x):
        x = self.relu(self.batchnorm_a(self.conv_a(x)))
        # x = self.max_pool_a(x)
        # x = self.dropout(x)
        x = self.relu(self.batchnorm_b(self.conv_b(x)))
        # x = self.max_pool_prime(x)
        x = self.dropout(x)
        x = self.relu(self.batchnorm_c(self.conv_c(x)))
        # x = self.max_pool_prime(x)
        # x = self.dropout(x)
        x = self.relu(self.batchnorm_d(self.conv_d(x)))
        # x = self.max_pool_prime(x)
        # x = self.dropout(x)
        x = self.relu(self.batchnorm_e(self.conv_e(x)))
        # x = self.max_pool_prime(x)
        x = self.dropout(x)
        x = x.max(-1)[0]
        # x = self.softmax(x)
        # x = torch.flatten(x,1)
        # x = self.fc1(x)
        # x = self.fc2(x)
        x = self.fc_lin(x)
        # x = F.log_softmax(x, dim=1) #at the moment the softmax is bad
        # x = self.avg_pool(x)
        # x = self.max_pool(x)

        return x

class BestperformingConvNet(nn.Module):

    def __init__(self):
        super(BestperformingConvNet, self).__init__()
        self.conv_a = nn.Conv1d(1, 8, 5, stride=5)
        self.batchnorm_a = nn.BatchNorm1d(8, eps=0.001, momentum=0.99)
        self.conv_b = nn.Conv1d(8, 32, 5, stride=3)
        self.batchnorm_b = nn.BatchNorm1d(32, eps=0.001, momentum=0.99)
        self.conv_c = nn.Conv1d(32, 64, 3, stride=3)
        self.batchnorm_c= nn.BatchNorm1d(64, eps=0.001, momentum=0.99)
        self.conv_d = nn.Conv1d(64, 128, 3, stride=2)
        self.batchnorm_d= nn.BatchNorm1d(128, eps=0.001, momentum=0.99)
        self.conv_e = nn.Conv1d(128, 256, 3, stride=1)
        self.batchnorm_e= nn.BatchNorm1d(256, eps=0.001, momentum=0.99)

        self.max_pool_a = nn.MaxPool1d(5, stride=1)
        self.max_pool_prime = nn.MaxPool1d(3, stride=1)
        self.dropout = nn.Dropout(p=0.2)
        # Size of a layer after convolution : (-W - F + 2P)/S +1 
        # / size of conv a after max pool : (8 - 2)/1 +1 = 7
        # self.avg_pool=torch.nn.AvgPool1d(kernel_size=256)

        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(117760, 128)
        self.fc2 = nn.Linear(128, 5)
        self.fc_lin = nn.Linear(256,5)

    def forward(self, x):

        x = self.relu(self.batchnorm_a(self.conv_a(x)))
        x = self.dropout(x)
        x = self.relu(self.batchnorm_b(self.conv_b(x)))
        # x = self.dropout(x)
        x = self.relu(self.batchnorm_c(self.conv_c(x)))
        # x = self.dropout(x)
        x = self.relu(self.batchnorm_d(self.conv_d(x)))
        x = self.dropout(x)
        x = self.relu(self.batchnorm_e(self.conv_e(x)))
        x = x.max(-1)[0]
        x = self.fc_lin(x)
        # x = F.log_softmax(x, dim=1) #at the moment the softmax is bad

        return x


class SpectrogramCNN(nn.Module):

    def __init__(self):
        super(SpectrogramCNN, self).__init__()
        self.conv_a = nn.Conv2d(1, 8, 25, stride=3)
        self.conv_b = nn.Conv2d(8, 16, 15, stride=2)
        self.conv_c = nn.Conv2d(16, 64, 5, stride=1)
        self.conv_d = nn.Conv2d(64, 128, 2, stride=1)
        self.conv_e = nn.Conv2d(128, 256, 2, stride=1)
       
        self.MP_prime= nn.MaxPool2d(5, stride=1)
        self.MP_ultra = nn.MaxPool2d(5, stride = 5)
        
        self.dropout1 = nn.Dropout(p=0.2)
        self.dropout2 = nn.Dropout(p=0.2)
        # Size of a layer after convolution : (-W - F + 2P)/S +1 
        # / size of conv a after max pool : (8 - 2)/1 +1 = 7
        # self.avg_pool=torch.nn.AvgPool1d(kernel_size=256)
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(19200, 128)
        self.fc2 = nn.Linear(128, 5)

    def forward(self, x):
        transf = T.Spectrogram(n_fft=528, win_length=None, hop_length=int(EEG_FS * 150e-3), center=True, pad_mode="reflect",power=2.0)
        x = transf(x)
        x = self.conv_a(x)
        x = self.relu(x)
        # x = self.MP_prime(x)
        x = self.dropout1(x)
        x = self.conv_b(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.MP_prime(x)
        x = self.relu(self.conv_c(x))
        x = self.dropout1(x)
        x = self.relu(self.conv_d(x))
        x = self.MP_ultra(x)
        # x = x.max(-1)[0]

        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x

def make_data_set(record):
    if type(record) == list:
        data = np.load(f'{directory}training_records/{record[0]}')
        record_number = int(record[0][-5])
        hypnogram = list(hypnograms[hypnograms['record'] == record_number]['target'])
        record = [record[0]]
        for r in range(1,len(record)):
            record_number = int(record[r][-5])
            h = list(hypnograms[hypnograms['record'] == record_number]['target'])
            hypnogram.extend(h)
            d = np.load(f'{directory}training_records/{record[r]}')
            data = np.vstack((data,d))
            record.append(record[r])
    else:
        data = np.load(f'{directory}training_records/{record}')
        record_number = int(record[-5])
        hypnogram = list(hypnograms[hypnograms['record'] == record_number]['target'])
        record = [record]
    return(data,hypnogram)

def make_data_set_test(record):
    if type(record) == list:
        data = np.load(f'{directory}test_records/{record[0]}')
        record_number = int(record[0][-5])
        record = [record[0]]
        for r in range(1,len(record)):
            record_number = int(record[r][-5])
            d = np.load(f'{directory}test_records/{record[r]}')
            data = np.vstack((data,d))
            record.append(record[r])
    else:
        data = np.load(f'{directory}test_records/{record}')
        record = [record]
    return(data)

def EEG_ACC(data) :
    EEG = data[:,1:EEG_FS * epoch_s * n_EEG + 1]
    EEG = EEG.reshape(len(data), n_EEG, EEG_FS * epoch_s)
    ACC = data[:,EEG_FS * epoch_s * n_EEG + 1:]
    ACC = ACC.reshape(len(data), n_ACC, ACC_FS * epoch_s)
    return EEG, ACC

def EEG_spectral(EEG) :
    EEG_spectral_power = spectral_power(EEG,n_EEG,EEG_FS)
    return EEG_spectral_power

def ACC_statistics(ACC):
    return only_stats(ACC,n_ACC)

def image_spectrale(EEG, eeg_i) :
    EEG_i = EEG[:,eeg_i,:]
    images = [0]*len(EEG_i)
    for i,eeg in enumerate(EEG_i) :
        fig = plot_spectrogram(np.clip(eeg,-200,200), 250, cmap='Spectral_r', win_sec = 5,trimperc = 2)
        fig.canvas.draw ()
        mat = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        mat = mat.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        images[i] = mat
        plt.close()
    return np.array(images)
    
class spectral_img(BaseEstimator, TransformerMixin) :
    def __init__(self,EEG_i = 0) :
        self.EEG_i = EEG_i 

    def fit(self, x, y) :
        return self

    def transform(self, x) :
        EEG,_ = EEG_ACC(x)
        self.img = image_spectrale(EEG, self.EEG_i)
        n =self.img.shape[1]*self.img.shape[2]*self.img.shape[3]
        return self.img.reshape(self.img.shape[0], n).astype('float32')

class EEG_unique(BaseEstimator, TransformerMixin) : 
    def __init__(self,EEG_i = 0) :
        self.EEG_i = EEG_i 

    def fit(self, x, y) :
        return self

    def transform(self, x) :
        EEG,_ = EEG_ACC(x)
        return EEG[:,self.EEG_i,:]

class spectral_and_acc(BaseEstimator, TransformerMixin) :
    def __init__(self) :
        return None

    def fit(self, x, y) :
        return self

    def transform(self, x) :
        EEG, ACC = EEG_ACC(x)
        self.EEG_spectral_power = EEG_spectral(EEG)
        self.ACC_stats = ACC_statistics(ACC)
        EEG_and_ACC_spectral_power = np.vstack(((self.EEG_spectral_power).T, (self.ACC_stats).T))
        return EEG_and_ACC_spectral_power.T


class CNN_code(ClassifierMixin, BaseEstimator) :

    def __init__(self):
        self.net = SingleChannelConvNet()
        # device: use GPU if available
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.net = self.net.to(self.device) # model into GP
        self.n_epoch =100
        self.min_validation_loss = np.inf
        self.min_validation_error = np.inf
        self.n_splits = 10
        self.learning_rate = 1e-3
        
    
    def fit(self, x, y) :
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.net.parameters())
        x, y = check_X_y(x, y)
        self.classes_ = unique_labels(y)
        self.X_ = x
        self.y_ = y
        self.net.train()
        print('training...')

        #KFold setting
        KCV = KFold(n_splits=self.n_splits) #shuffle=True, random_state=15011999 
        self.val_error = []
        self.training_error = []
        for fold, (train_index, val_index) in enumerate(KCV.split(x)):
            print('------------fold no---------{}----------------------'.format(fold+1))
            # weight resetting: avoid weight leakage
            self.net.apply(weights_init)

            early_stopper = EarlyStopper(patience=5, min_delta=0.5)

            train_subsampler = torch.utils.data.SubsetRandomSampler(train_index)
            val_subsampler = torch.utils.data.SubsetRandomSampler(val_index)
            
            training_dataset = EegEpochDataset(x,y)
            training_dataloader = DataLoader(training_dataset, sampler=train_subsampler, batch_size = 32)
            validation_dataloader = DataLoader(training_dataset, sampler=val_subsampler, batch_size = 32)

            
            for epoch in range(self.n_epoch):  # loop over the dataset multiple times

                running_loss = 0.0
                prediction_list = torch.empty(0).to(self.device)
                true_list = torch.empty(0).to(self.device)
                for i, data in enumerate(training_dataloader, 0):
                    # get the inputs; data is a list of [inputs, labels]
                    inputs, labels = data
                    # torch.squeeze(inputs, dim=1)
                    # print(inputs.shape)
                    inputs, labels = inputs.to(self.device).float(), labels.to(self.device)

                    # zero the parameter gradients
                    optimizer.zero_grad()
                    # forward + backward + optimize
                    outputs = self.net.forward(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    # training f1
                    _, predicted = torch.max(outputs, 1)
                    prediction_list = torch.cat([prediction_list, predicted])
                    true_list = torch.cat([true_list, labels])

                true_list = true_list.cpu().numpy()
                prediction_list = prediction_list.cpu().numpy()

                train_f1 = f1_score(true_list, prediction_list, average = 'macro')
                train_error = 1-train_f1
                self.training_error.append(train_error)

                    # print statistics
                print('epoch %d, %d samples, loss: %.3f' % (epoch + 1, (i+1)*training_dataloader.batch_size,running_loss / (i+1)), end = ", ")
                print('training f1: %.3f' % (train_f1), end = ', ')
                
                running_loss = 0.0

                #VALIDATION
                with torch.no_grad():
                    self.net.eval()
                    validation_loss = 0.0
                    prediction_list = torch.empty(0).to(self.device)
                    true_list = torch.empty(0).to(self.device)
                    for i, data in enumerate(validation_dataloader, 0):
                        # get the inputs; data is a list of [inputs, labels]
                        inputs, labels = data
                        inputs, labels = inputs.to(self.device).float(), labels.to(self.device)

                        # forward
                        outputs = self.net.forward(inputs)
                        loss = criterion(outputs, labels)
                        
                        validation_loss += loss.item()
                        # evaluate f1 validation
                        _, predicted = torch.max(outputs, 1)
                        prediction_list = torch.cat([prediction_list, predicted])
                        true_list = torch.cat([true_list, labels])
                    
                    true_list = true_list.cpu().numpy()
                    prediction_list = prediction_list.cpu().numpy()

                    validation_f1 = f1_score(true_list, prediction_list, average = 'macro')
                    validation_error = 1-validation_f1
                    self.val_error.append(validation_error)
                    
                    # print statistics
                print('validation loss: %.3f' % (validation_loss / (i+1)), end=', ')
                print('validation f1: %.3f' % (validation_f1))

                if validation_error < (self.min_validation_error):
                    self.min_validation_error = validation_error
                    print('new minimal validation error')
                    torch.save(self.net.state_dict(), 'opti_test')

                if validation_loss < self.min_validation_loss:
                    self.min_validation_loss = validation_loss
                    torch.save(self.net.state_dict(), 'loss_test')

                
                if early_stopper.early_stop(validation_loss):    
                    print('aie at epoch', epoch)         
                    break
                else:
                    torch.save(self.net.state_dict(), 'my_net_opti')

            
            print('saved error so far: ', self.min_validation_error)

        return(self)

    def predict(self, x) :
        check_is_fitted(self, ['X_', 'y_'])
        x = check_array(x)
        test_dataloader = DataLoader(EegEpochDataset_test(x))
        with torch.no_grad():
            prediction_list = torch.empty(0).to(self.device)
            for data in test_dataloader:
                inputs = data
                inputs = inputs.to(self.device).float()
                outputs = self.net(inputs)
                _, predicted = torch.max(outputs, 1)
                prediction_list = torch.cat([prediction_list, predicted])

        return prediction_list.tolist()
    
    def predict_proba(self, x) :
        check_is_fitted(self, ['X_', 'y_'])
        x = check_array(x)
        test_dataloader = DataLoader(EegEpochDataset_test(x))
        with torch.no_grad():
            prediction_list = torch.empty(0).to(self.device)
            for data in test_dataloader:
                inputs = data
                inputs = inputs.to(self.device).float()
                outputs = self.net(inputs)
                prediction_list = torch.cat([prediction_list, outputs])

        return prediction_list.tolist()


def weights(n) :
    l = [rd.uniform(0,1) for i in range (n-1)]
    l.sort()
    l = [0] + l 
    l.append(1)
    return([l[i] - l[i-1] for i in range (1,len(l))])


def scoring(y,pred) :
    ConfusionMatrixDisplay.from_predictions(y,pred)
    plt.show()
    print({'balanced_accuracy': balanced_accuracy_score(y,pred),
            'cohen_kappa': cohen_kappa_score(y,pred),
            'macro_f1': f1_score(y,pred,average ='macro')})
    

Useful preprocessing (using fit_transform(x))
- Normalizer()
- MinMaxScaler()
- TruncatedSVD()
- PCA(random_state=42, svd_solver=solver, n_components=n)  (solver = 'arpack' or 'auto')
- FastICA(random_state=42, n_components=n, whiten=False) if whiten False use whitening function

In [30]:
types = 'train', 'test'
records_list = listdir(directory + "training_records")
records = {}
records[types[0]], records[types[1]] = split_train_test(records_list, split=0.2)
x = {}
y = {}
for t in types:
    x[t], y[t]= make_data_set(records[t])

test_records = listdir(directory + "test_records")

In [31]:
models_spectral = 'random_forest', 'SVM', 'KNN', 'multi_layer_perceptron', 'decision_tree'
models_funct = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10), \
        SVC(random_state = 42, max_iter=150, probability = True), \
        KNeighborsClassifier(n_neighbors=10, weights='distance'), \
        MLPClassifier(activation='tanh', max_iter=300), \
        DecisionTreeClassifier(max_depth=15)
available_cnns = SpectrogramCNN(), BestperformingConvNet(), SingleChannelConvNet()

pipelines = []

# FOR OTHER MODELS WITH SPECTRAL ANALYSIS AS INPUT 

for m,model in enumerate(models_spectral):
    pip = Pipeline([('spectral_analysis', spectral_and_acc()), ('scale', MinMaxScaler()), (model, models_funct[m])])
    pipelines.append((f'pipe_{model}', pip))

# FOR SINGLE CHANNEL SPECTRAL CNN 
for i in range (n_EEG) :
    pip = Pipeline([('spectro', EEG_unique(EEG_i =i)), ('cnn', CNN_code())])
    #pip.fit(x['train'], y['train'])
    pipelines.append((f'pipe_cnn_{i}', pip))

l = len(pipelines)
vc = VotingClassifier(estimators=pipelines, voting='soft',n_jobs=-1, weights = weights(l))
n=3

weights_grid = [weights(l) for _ in range (n)]
for i in range (n) :
    weights_grid.append(weights(l))
weights_grid = {'weights':weights_grid}
grid_search = GridSearchCV(param_grid=weights_grid,estimator = vc, cv=n, n_jobs=-1, verbose=10, scoring="f1_macro")
grid_search.fit(x['train'], y['train'])
print(grid_search.best_score_)

# for i in range (n) :
#     a = int((i)*x['train'].shape[0]/n) 
#     b = int((i+1)*x['train'].shape[0]/n) -1
#     vc.set_params(weights = weights(l))
#     vc.fit(x['train'][a:b], y['train'][a:b])
#     pred = vc.predict(x['test'])
#     scoring(y['test'], pred)
#     filename = f"{out_directory}votingclass_{i}.joblib.pkl"
#     joblib.dump(vc, filename, compress=9)
    


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3; 1/6] START weights=[0.014618780486909122, 0.04994350049027696, 0.049850688711501556, 0.03414166901959992, 0.03453600870164408, 0.2828079115983779, 0.020852981596448772, 0.054336644946371915, 0.42381337536508545, 0.035098439083784294]
training...
------------fold no---------1----------------------
training...
------------fold no---------1----------------------
epoch 1, 512 samples, loss: 4.058, training f1: 0.257, epoch 1, 512 samples, loss: 3.282, training f1: 0.297, validation loss: 1.472, validation f1: 0.296
new minimal validation error
validation loss: 1.172, validation f1: 0.225
new minimal validation error
training...
------------fold no---------1----------------------
training...
------------fold no---------1----------------------
[CV 1/3; 1/6] END weights=[0.014618780486909122, 0.04994350049027696, 0.049850688711501556, 0.03414166901959992, 0.03453600870164408, 0.2828079115983779, 0.020852981596448772, 0.05433

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
18 fits failed out of a total of 18.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/_voting.py", line 324, in fit
    return super().fit(X, transformed_y, sample_weight)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/_voting.py", line 83, in fit
    for idx, clf in enumerate(clfs)
  File "/opt/conda/lib/python3.7/site-p

OSError: [Errno 30] Read-only file system: 'opti_test'

In [ ]:
predictions = []
X_size = 0
pred_size = 0
for test in test_records:
    X_test = make_data_set_test(test)
    size, _ = X_test.shape
    X_size += size
    prediction_list = weights_grid.pred(X_test)
    pred_size += len(prediction_list)
    record_number = int(test[-5])
    for i, pred in enumerate(prediction_list):
        predictions.append({"identifier":record_number * 10000 + i,'target':int(pred)})
    


predictions = pd.DataFrame(predictions)
print(predictions)
predictions.to_csv('submission.csv',index = None)

if len(predictions) != 2646:
    print('WARNING: incorrect submission length! (', len(predictions), ' instead of 2646)')
else:
    print('GOOD TO GO! CORRECT SIZE')

https://github.com/Kaggle/kaggle-api